# Asistente Financiero con RAG y Tools

## A) Instalación de dependencias

En tu Jupyter Notebook (o terminal) ejecuta:

```bash
pip install \
  langchain[pypdf] \        # LangChain + soporte PDF (PyPDFLoader)
  openai \                  # Cliente para modelos de OpenAI
  tiktoken \                # Tokenizador necesario para embeddings
  faiss-cpu \               # Índice vectorial FAISS en CPU
  gradio \                  # Framework para interfaz web
  requests \                # Llamadas HTTP (API de conversión de divisas)
  pypdf \                   # Librería de lectura de PDF (por si falla el extra)
  unstructured \            # Loader alternativo para PDFs complejos
  python-dotenv             # Para cargar variables desde .env


# B) Arquitectura de la Solución

La siguiente arquitectura muestra cómo se orquestan los distintos componentes para ofrecer:

1. **Preguntas sobre PDF** mediante RAG (Retrieval-Augmented Generation).  
2. **Conversión de divisas** de forma independiente.  

---

## 1. Capa de Presentación (Frontend)

- **Gradio Blocks** embebido en Jupyter o servidor local.  
- Dos secciones claramente separadas:  
  1. **Consultas al Informe Financiero (PDF)**  
     - Botón **Cargar PDF**  
     - Cuadro de texto y botón **Enviar consulta PDF**  
     - Caja de texto para el estado del PDF  
  2. **Conversión de Moneda**  
     - Cuadro de texto para la frase de conversión (p.ej. `150 EUR a USD`)  
     - Botón **Convertir**  
     - Caja de texto para el resultado de la conversión  

---

## 2. Capa de Orquestación (Backend)

- **Controlador Gradio**  
  - Captura eventos de los botones y mapea a las funciones Python:  
    - `load_pdf`  
    - `answer_pdf_query`  
    - `answer_conversion_query`  
- **Estado Global**  
  - `vectorstore` y `qa_chain` se inicializan tras cargar el PDF.  
  - La herramienta de conversión es independiente de ese estado y siempre está disponible.

---

## 3. Pipeline RAG para QA sobre PDF

1. **Document Loader**  
   - `PyPDFLoader` o `UnstructuredPDFLoader` fragmenta el PDF en trozos semánticos.  
2. **Embeddings**  
   - `OpenAIEmbeddings` convierte cada fragmento en un vector de punto flotante.  
3. **Vector Store**  
   - `FAISS` indexa localmente todos los vectores para búsqueda de similitud.  
4. **RetrievalQA Chain**  
   - Al recibir una pregunta:  
     - Recupera los fragmentos más relevantes de FAISS.  
     - Pasa esos fragmentos a `ChatOpenAI` para generar la respuesta final.

---

## 4. Servicio de Conversión de Moneda

- **Parser Regex** en `answer_conversion_query` detecta patrones:  

## C) Mejoras Futuras

- **Preprocesamiento avanzado**: soportar OCR (PDF escaneados) y extraer metadatos (fecha, autor).  
- **UI más rica**: añadir historial de consultas y chat continuo para contextualizar preguntas.  
- **Persistencia del índice**: guardar el vectorstore en disco o en un servicio gestionado para evitar reindexar.  
- **Nuevas herramientas**: incorporar calculadora financiera y generación de gráficos dinámicos.  
- **Optimización y caché**: batch de embeddings, caching de conversiones de divisas y límites de tamaño de archivo.  


In [2]:
import sys
import subprocess

def silent_install(package):
    try:
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", package],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL
        )
        print(f"✅ {package} instalado (o ya estaba).")
    except Exception as e:
        print(f"❌ Error instalando {package}: {e}")

# Lista de paquetes
packages = [
    "openai",
    "faiss-cpu",
    "gradio",
    "requests",
    "pypdf",
    "langchain-community",
    "tiktoken",
    "langchain[pypdf]"
]

for pkg in packages:
    silent_install(pkg)


✅ openai instalado (o ya estaba).
✅ faiss-cpu instalado (o ya estaba).
✅ gradio instalado (o ya estaba).
✅ requests instalado (o ya estaba).
✅ pypdf instalado (o ya estaba).
✅ langchain-community instalado (o ya estaba).
✅ tiktoken instalado (o ya estaba).
✅ langchain[pypdf] instalado (o ya estaba).


In [1]:
import os
import re
import requests
import gradio as gr
from dotenv import load_dotenv

# Carga variables de entorno desde .env (si existe)
if os.path.exists(".env"):
    load_dotenv(".env")

# Compatibilidad con loaders de PDF
try:
    from langchain.document_loaders import PyPDFLoader as PDFLoader
except ImportError:
    from langchain.document_loaders import UnstructuredPDFLoader as PDFLoader

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# Inicializar LLM y embeddings
env_api_key = os.getenv("OPENAI_API_KEY") or "OPENAI_API_KEY"
llm = ChatOpenAI(openai_api_key=env_api_key, temperature=0)
embeddings = OpenAIEmbeddings(openai_api_key=env_api_key)

# Variables globales para PDF QA
global vectorstore, qa_chain
vectorstore = None
qa_chain = None

# -------------------------------------
# Tool de conversión de divisas (no requiere API key)
def convert_currency(amount: float, from_currency: str, to_currency: str) -> str:
    """Convierte amount de from_currency a to_currency usando open.er-api.com (sin API key)"""
    url = f"https://open.er-api.com/v6/latest/{from_currency.upper()}"
    try:
        resp = requests.get(url, timeout=10)
        resp.raise_for_status()
        data = resp.json()
        if data.get("result") != "success":
            return f"❌ API error: {data.get('error-type', data)}"
        rates = data.get("rates", {})
        rate = rates.get(to_currency.upper())
        if rate is None:
            return f"❌ Tasa no encontrada para {to_currency.upper()}. Disponible: {list(rates.keys())[:5]}..."
        converted = rate * amount
        return f"{amount:.2f} {from_currency.upper()} = {converted:.2f} {to_currency.upper()}"
    except Exception as e:
        return f"❌ Error en conversión: {e}"
# -------------------------------------

# Carga y preparación del PDF para RAG

def load_pdf(pdf_file):
    """Carga el PDF, construye vectorstore y la cadena RAG para QA"""
    global vectorstore, qa_chain #vectorstore: Indice vectorial del contenido del PDF, qa_chain: Cadena RAG configurada para responder preguntas.
    try:
        loader = PDFLoader(pdf_file.name) # PDFLoader: Cargador de PDFs (LangChain).
        docs = loader.load_and_split() # Lee el PDF y Divide su contenido en fragmentos para poder vectorizarlos de forma eficiente. Obtiendo una lista de documentos.
        # FAISS: Es una librería de Facebook para búsquedas rápidas en vectores.
        # Aquí se convierten los fragmentos de texto en vectores usando los embeddings.
        # Luego, se almacenan los vectores en un índice FAISS para poder buscar contenido similar al hacer preguntas.
        vectorstore = FAISS.from_documents(docs, embeddings)
        # Se crea una cadena RAG (Retrieval-Augmented Generation).
        # llm=llm: Modelo de lenguaje.
        # chain_type="stuff": Tipo básico de cadena donde se "inyectan" los textos recuperados directamente al LLM.
        # retriever=vectorstore.as_retriever(): Conectas el índice vectorial para que, al recibir una pregunta, busque los fragmentos relevantes del PDF.
        # return_source_documents=False: Solo devuelve la respuesta, sin mostrar los documentos fuente.
        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=vectorstore.as_retriever(),
            return_source_documents=False
        )
        return "📄 PDF cargado y listo para consultas"
    except Exception as exc:
        return f"❌ Error al cargar PDF: {exc}"

# Funciones de respuesta separadas

def answer_pdf_query(query: str) -> str:
    """Responde preguntas sobre el PDF usando RAG"""
    if qa_chain is None:
        return "⚠️ Primero carga un PDF antes de preguntar al informe."
    try:
        return qa_chain.run(query)
    except Exception as e:
        return f"❌ Error al generar respuesta PDF: {e}"


def answer_conversion_query(query: str) -> str:
    """Responde conversiones de moneda independientemente del PDF"""
    # Regex: cantidad + 3-letras + 'a'|'en' + 3-letras
    match = re.search(r"([0-9]+(?:\.[0-9]+)?)\s*([A-Za-z]{3})\s+(?:a|en)\s+([A-Za-z]{3})", query)
    if match:
        amt, frm, to = match.groups()
        try:
            return convert_currency(float(amt), frm, to)
        except Exception as e:
            return f"❌ Error interno conversión: {e}"
    return "⚠️ Usa formato: '<monto> <MONEDA_ORIGEN> a|en <MONEDA_DESTINO>' (p.ej. '100 USD a EUR')"

# Construcción de la interfaz Gradio
demo = gr.Blocks()
with demo:
    gr.Markdown("""
                  <center><h1>Banco Guayaquil</h1></center>
                  <center><h3>Asistente Financiero: PDF + Conversión de Moneda</h3></center>
               """)

    # Sección PDF QA
    gr.Markdown("## Consultas al Informe Financiero (PDF)")
    with gr.Row():
        pdf_input = gr.File(label="Sube tu informe financiero (PDF)")
        pdf_load_btn = gr.Button("Cargar PDF")
    status = gr.Textbox(label="Estado PDF", interactive=False)
    pdf_query = gr.Textbox(label="Pregunta", lines=2, placeholder="Pregunta al informe... (p.ej. 'Total de activos 2021')")
    pdf_submit = gr.Button("Enviar consulta PDF")
    pdf_output = gr.Textbox(label="Respuesta PDF", interactive=False)

    pdf_load_btn.click(fn=load_pdf, inputs=pdf_input, outputs=status)
    pdf_submit.click(fn=answer_pdf_query, inputs=pdf_query, outputs=pdf_output)

    # Sección Conversión de Moneda
    gr.Markdown("## Conversión de Moneda")
    conv_query = gr.Textbox(label="Pregunta", lines=1, placeholder="Ej. '150 EUR a USD'")
    conv_btn = gr.Button("Convertir")
    conv_output = gr.Textbox(label="Resultado Conversión", interactive=False)

    conv_btn.click(fn=answer_conversion_query, inputs=conv_query, outputs=conv_output)

# Lanzar en Jupyter / Notebook con share URL
demo.launch(share=True)


<ipython-input-1-7c1342bc9814>:24: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(openai_api_key=env_api_key, temperature=0)
<ipython-input-1-7c1342bc9814>:25: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=env_api_key)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ca8388c25bec55be9e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
